In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error


In [ ]:
from currency_converter import CurrencyConverter
c = CurrencyConverter(fallback_on_missing_rate=True)


In [ ]:
# load dataset

df_train= pd.read_csv(".\Train_TP2_Datos_2020-2C.csv") #train
df_test = pd.read_csv(".\Test_TP2_Datos_2020-2C.csv")  #test

In [ ]:
#PARTE 1 - Limpieza y Featuring Engineering

In [ ]:
df_train = df_train[df_train["Stage"].str.contains("Closed")]

In [ ]:
#Armo Target y limpio columnas

df_train = df_train.drop(columns=["Sales_Contract_No", "Product_Type", "Brand", "Opportunity_Name", "Last_Modified_By", "Billing_Country", "Size", "ID", "Prod_Category_A", 'Product_Category_B', "Actual_Delivery_Date" ])
df_test = df_test.drop(columns=["Sales_Contract_No", "Product_Type", "Brand", "Opportunity_Name", "Last_Modified_By", "Billing_Country", "Size", "ID", "Prod_Category_A", 'Product_Category_B', "Actual_Delivery_Date" ])

df_train['Last_Modified_Date']=pd.to_datetime(df_train['Last_Modified_Date'])
df_test['Last_Modified_Date']=pd.to_datetime(df_test['Last_Modified_Date'])

In [ ]:
#PROCESO COLUMNAS DE TIEMPOS, FECHAS Y PRECIOS

df_train["Opportunity_Created_Date"] = pd.to_datetime(df_train["Opportunity_Created_Date"], errors="coerce")
df_train["Account_Created_Date"] = pd.to_datetime(df_train["Account_Created_Date"], errors="coerce")
df_train["Quote_Expiry_Date"] = pd.to_datetime(df_train["Quote_Expiry_Date"], errors="coerce")
df_train["Last_Modified_Date"] = pd.to_datetime(df_train["Last_Modified_Date"], errors="coerce")

df_train['Total_Taxable_Amount_(converted)']= df_train.apply(lambda x: c.convert(x.Total_Taxable_Amount, x.Total_Taxable_Amount_Currency, 'USD', date=x.Last_Modified_Date), axis=1)

df_train["Month"] = pd.to_datetime(df_train["Month"], format = "%Y - %m")
df_train["Month Delivery"] = df_train["Month"].apply(lambda x: int(x.month))

df_train["Month Creation"] = df_train["Opportunity_Created_Date"].apply(lambda x: int(x.month))

df_train["Planned_Delivery_Start_Date"] = pd.to_datetime(df_train["Planned_Delivery_Start_Date"], errors="coerce")
df_train["Planned_Delivery_End_Date"] = pd.to_datetime(df_train["Planned_Delivery_End_Date"], errors="coerce")
delay = df_train['Planned_Delivery_Start_Date'].sub(df_train['Opportunity_Created_Date'], axis=0)/ np.timedelta64(1, 'D')
df_train["Delivery_Delay"] = [x if x > 0 else 0 for x in delay]


duracionPresupuesto = df_train['Quote_Expiry_Date'].sub(df_train['Opportunity_Created_Date'], axis=0)/ np.timedelta64(1, 'D')
df_train["duracionPresupuesto"] = [x if x > 0 else 0 for x in duracionPresupuesto]

duration = df_train['Last_Modified_Date'].sub(df_train['Opportunity_Created_Date'], axis=0)/ np.timedelta64(1, 'D')
df_train["duration"] = [x if x > 0 else 0 for x in duration]

timeInSystem = np.floor(df_train["Opportunity_Created_Date"].sub(df_train["Account_Created_Date"], axis=0)/ np.timedelta64(1, 'D'))
df_train["timeInSystem"] = [x if x > 0 else 0 for x in timeInSystem]

fromDate1 = min(df_train['Opportunity_Created_Date'])
fromDate2 = min(df_train['Last_Modified_Date'])
fromDate3 = min(df_train['Account_Created_Date'])

df_train['Opportunity_Created_Date_delta'] = (df_train["Opportunity_Created_Date"] - fromDate1).dt.days.astype(int)

df_train.drop(columns= ["Submitted_for_Approval", "Opportunity_Created_Date", "Total_Amount",\
                         "Planned_Delivery_Start_Date","Planned_Delivery_End_Date", \
                         "Quote_Expiry_Date", "Last_Modified_Date", "Account_Created_Date", "Total_Taxable_Amount_Currency", \
                        "Total_Taxable_Amount", "ASP", "ASP_(converted)_Currency", \
                        "ASP_Currency", "Currency"], inplace = True)

#-----------------------------------------------
#-----------------------------------------------
#-----------------------------------------------
#-----------------------------------------------


df_test["Opportunity_Created_Date"] = pd.to_datetime(df_test["Opportunity_Created_Date"], errors="coerce")
df_test["Account_Created_Date"] = pd.to_datetime(df_test["Account_Created_Date"], errors="coerce")
df_test["Quote_Expiry_Date"] = pd.to_datetime(df_test["Quote_Expiry_Date"], errors="coerce")
df_test["Last_Modified_Date"] = pd.to_datetime(df_test["Last_Modified_Date"], errors="coerce")

df_test['Total_Taxable_Amount_(converted)']= df_test.apply(lambda x: c.convert(x.Total_Taxable_Amount, x.Total_Taxable_Amount_Currency, 'USD', date=x.Last_Modified_Date), axis=1)


df_test["Month"] = pd.to_datetime(df_test["Month"], format = "%Y - %m")
df_test["Month Delivery"] = df_test["Month"].apply(lambda x: int(x.month))

df_test["Month Creation"] = df_test["Opportunity_Created_Date"].apply(lambda x: int(x.month))

df_test["Planned_Delivery_Start_Date"] = pd.to_datetime(df_test["Planned_Delivery_Start_Date"], errors="coerce")
df_test["Planned_Delivery_End_Date"] = pd.to_datetime(df_test["Planned_Delivery_End_Date"], errors="coerce")
delay = df_test['Planned_Delivery_Start_Date'].sub(df_test['Opportunity_Created_Date'], axis=0)/ np.timedelta64(1, 'D')
df_test["Delivery_Delay"] = [x if x > 0 else 0 for x in delay]

duracionPresupuesto = df_test['Quote_Expiry_Date'].sub(df_test['Opportunity_Created_Date'], axis=0)/ np.timedelta64(1, 'D')
df_test["duracionPresupuesto"] = [x if x > 0 else 0 for x in duracionPresupuesto]

duration = df_test['Last_Modified_Date'].sub(df_test['Opportunity_Created_Date'], axis=0)/ np.timedelta64(1, 'D')
df_test["duration"] = [x if x > 0 else 0 for x in duration]

timeInSystem = np.floor(df_test["Opportunity_Created_Date"].sub(df_test["Account_Created_Date"], axis=0)/ np.timedelta64(1, 'D'))
df_test["timeInSystem"] = [x if x > 0 else 0 for x in timeInSystem]

df_test['Opportunity_Created_Date_delta'] = (df_test["Opportunity_Created_Date"] - fromDate1).dt.days.astype("int64")

df_test.drop(columns= [ "Submitted_for_Approval", "Opportunity_Created_Date", "Total_Amount",\
                        "Planned_Delivery_Start_Date","Planned_Delivery_End_Date", \
                         "Quote_Expiry_Date", "Last_Modified_Date", "Account_Created_Date", "Total_Taxable_Amount_Currency", \
                        "Total_Taxable_Amount", "ASP", "ASP_(converted)_Currency", \
                        "ASP_Currency", "Currency"], inplace = True)

In [ ]:
#Agrupo por opportunity
aux1 = df_train.groupby(["Opportunity_ID"]).agg({"Territory": "count", "ASP_(converted)": ["sum", "mean"], "TRF": ["sum", "mean", "max"]}).reset_index()
aux2 = df_test.groupby(["Opportunity_ID"]).agg({"Territory": "count", "ASP_(converted)": ["sum", "mean"], "TRF": ["sum", "mean", "max"]}).reset_index()

aux1.columns = ["Opportunity_ID", "Products_Quantity", "ASP sum", "ASP mean", "Total TRF", "TRF mean", "TRF max"]
aux2.columns = ["Opportunity_ID", "Products_Quantity", "ASP sum", "ASP mean", "Total TRF", "TRF mean", "TRF max"]
df_train = df_train.drop_duplicates(subset="Opportunity_ID")
df_test = df_test.drop_duplicates(subset="Opportunity_ID")

df_train = df_train.merge(aux1, on="Opportunity_ID")
df_test = df_test.merge(aux2, on="Opportunity_ID")

opps = df_train.groupby(["Account_Name"]).agg({"Territory":"count"}).reset_index()
opps.columns=["Account_Name", "Total Opportunities"]
df_train.merge(opps,on="Account_Name", how="right")


df_train = df_train.drop(columns=["TRF", "ASP_(converted)"])
df_test = df_test.drop(columns=["TRF", "ASP_(converted)"])


In [ ]:
df_train

,Region,Territory,"Pricing, Delivery_Terms_Quote_Appr","Pricing, Delivery_Terms_Approved",Bureaucratic_Code_0_Approval,Bureaucratic_Code_0_Approved,Bureaucratic_Code,Source,Account_Name,Opportunity_ID,...,duracionPresupuesto,duration,timeInSystem,Opportunity_Created_Date_delta,Products_Quantity,ASP sum,ASP mean,Total TRF,TRF mean,TRF max
0,EMEA,None,1,1,1,1,Bureaucratic_Code_4,None,Account_Name_619,0,...,54.0,189.0,174.0,762,1,0.58817,0.58817,10,10.0,10
1,EMEA,None,0,0,0,0,Bureaucratic_Code_4,None,Account_Name_619,1,...,44.0,39.0,174.0,762,1,0.59948,0.59948,0,0.0,0
2,Americas,NW America,0,0,0,0,Bureaucratic_Code_4,Source_7,Account_Name_1794,2,...,297.0,296.0,231.0,763,1,0.48000,0.48000,0,0.0,0
3,Americas,NW America,1,0,1,0,Bureaucratic_Code_5,Source_11,Account_Name_1201,3,...,0.0,840.0,864.0,763,1,0.53000,0.53000,14,14.0,14
4,Americas,NW America,1,0,1,0,Bureaucratic_Code_5,Source_11,Account_Name_1201,4,...,542.0,840.0,864.0,763,1,0.53000,0.53000,25,25.0,25
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9786,EMEA,Norway,1,1,0,0,Bureaucratic_Code_4,Source_9,Account_Name_533,12799,...,121.0,119.0,860.0,759,1,0.58251,0.58251,1,1.0,1
9787,Americas,NE America,1,0,0,0,Bureaucratic_Code_5,None,Account_Name_404,12800,...,42.0,311.0,536.0,759,2,1.06000,0.53000,40,20.0,20
9788,EMEA,Austria,1,1,1,1,Bureaucratic_Code_4,Source_7,Account_Name_726,12801,...,0.0,46.0,0.0,759,3,1.76451,0.58817,0,0.0,0
9789,Americas,NE America,1,1,1,1,Bureaucratic_Code_4,None,Account_Name_944,12802,...,86.0,48.0,46.0,760,1,0.63750,0.63750,4,4.0,4


In [ ]:
df_test

,Region,Territory,"Pricing, Delivery_Terms_Quote_Appr","Pricing, Delivery_Terms_Approved",Bureaucratic_Code_0_Approval,Bureaucratic_Code_0_Approved,Bureaucratic_Code,Source,Account_Name,Opportunity_ID,...,duracionPresupuesto,duration,timeInSystem,Opportunity_Created_Date_delta,Products_Quantity,ASP sum,ASP mean,Total TRF,TRF mean,TRF max
0,EMEA,Germany,1,1,1,1,Bureaucratic_Code_4,Source_7,Account_Name_978,10689,...,18.0,0.0,646.0,1984,3,1.04061,0.346870,0,0.0,0
1,EMEA,Germany,1,1,1,1,Bureaucratic_Code_4,Source_7,Account_Name_978,10690,...,18.0,13.0,646.0,1984,5,1.87762,0.375524,1,0.2,1
2,Americas,NE America,1,1,0,0,Bureaucratic_Code_4,Source_9,Account_Name_143,10691,...,0.0,4.0,55.0,1984,1,0.42500,0.425000,0,0.0,0
3,Americas,NW America,1,1,1,0,Bureaucratic_Code_2,Source_11,Account_Name_25,10692,...,0.0,4.0,2085.0,1984,6,2.55000,0.425000,6,1.0,1
4,Americas,NW America,0,0,0,0,Bureaucratic_Code_4,Source_9,Account_Name_1192,10693,...,30.0,1.0,225.0,1984,1,0.39000,0.390000,0,0.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1562,EMEA,KSA,1,1,0,0,Bureaucratic_Code_4,Source_3,Account_Name_1686,12364,...,28.0,0.0,683.0,1998,2,0.60000,0.300000,0,0.0,0
1563,Americas,SE America,1,1,0,0,Bureaucratic_Code_4,None,Account_Name_768,12365,...,28.0,1.0,322.0,1998,1,0.41000,0.410000,0,0.0,0
1564,Americas,SE America,1,1,1,1,Bureaucratic_Code_4,None,Account_Name_768,12366,...,28.0,1.0,322.0,1998,1,0.41000,0.410000,0,0.0,0
1565,Americas,NE America,1,1,0,0,Bureaucratic_Code_4,None,Account_Name_1635,12367,...,30.0,0.0,240.0,1998,1,0.50000,0.500000,0,0.0,0


In [ ]:
#Proceso las columnas categoricas
#MEJORAR A MEAN ENCODING???

from sklearn.preprocessing import OneHotEncoder

def crearLista (listadoCompleto):
    listaReducida = []
    for i in listadoCompleto:
        if i not in listaReducida:
            listaReducida.append(i)
    listaReducida.sort()
    return listaReducida

def discretizar(columna,nombre, df):
    listaReducida = crearLista(columna)
    v = list(range(len(columna)))
    listaCompleta = list(columna)
    for i in listaReducida:
        for j in range(len(listaCompleta)):
            if(listaCompleta[j] == i):
                v[j] = 1
            else:
                v[j] = 0
        df[nombre+str(i)] = v

discretizar(df_train.Region,'Region ', df_train)

#--------------
#MEAN
#
#df_train["Targetx"] = [1 if x == "Closed Won" else 0 for x in stage_train]
#df_train.sort_values(by="Opportunity_Created_Date_delta", ascending=False)

#cumulative_sum = df_train.groupby(["Region"])["Targetx"].cumsum() - df_train["Targetx"]
#cumulative_count = df_train.groupby(["Region"]).cumcount()
#df_train["Region_mean_target"] = cumulative_sum/cumulative_count
#df_train["Region_mean_target"].astype("float32")
#-----------
#df_train["Stage"] = df_train["Stage"].map(lambda x: 1 if (x == "Closed Won") else 0)

stage_train = df_train.Stage
df_train["Target"] = [1 if x == "Closed Won" else 0 for x in stage_train]
df_train = df_train.drop(columns=["Stage"])
df_train = df_train.fillna(0)

df_train = df_train.drop(columns=["Opportunity_ID", "Delivery_Year", "Month Delivery", "Month Creation", "duracionPresupuesto"])

toTrain = df_train.select_dtypes("number")

#-----------------------------------------------------------
#-----------------------------------------------------------
#-----------------------------------------------------------
#-----------------------------------------------------------
#-----------------------------------------------------------

discretizar(df_test.Region,'Region ', df_test)

df_test = df_test.fillna(0)

ids = df_test["Opportunity_ID"]
df_test = df_test.drop(columns=["Opportunity_ID", "Delivery_Year", "Month Delivery", "Month Creation", "duracionPresupuesto"])

toTest = df_test.select_dtypes("number")

In [ ]:
toTrain

,"Pricing, Delivery_Terms_Quote_Appr","Pricing, Delivery_Terms_Approved",Bureaucratic_Code_0_Approval,Bureaucratic_Code_0_Approved,Total_Taxable_Amount_(converted),Delivery_Delay,duration,timeInSystem,Opportunity_Created_Date_delta,Products_Quantity,...,ASP mean,Total TRF,TRF mean,TRF max,Region APAC,Region Americas,Region EMEA,Region Japan,Region Middle East,Target
0,1,1,1,1,5.941391e+06,146.0,189.0,174.0,762,1,...,0.58817,10,10.0,10,0,0,1,0,0,0
1,0,0,0,0,5.263822e+04,42.0,39.0,174.0,762,1,...,0.59948,0,0.0,0,0,0,1,0,0,1
2,0,0,0,0,8.386560e+04,48.0,296.0,231.0,763,1,...,0.48000,0,0.0,0,0,1,0,0,0,1
3,1,0,1,0,7.421882e+06,786.0,840.0,864.0,763,1,...,0.53000,14,14.0,14,0,1,0,0,0,0
4,1,0,1,0,1.335719e+07,786.0,840.0,864.0,763,1,...,0.53000,25,25.0,25,0,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9786,1,1,0,0,4.592234e+05,119.0,119.0,860.0,759,1,...,0.58251,1,1.0,1,0,0,1,0,0,1
9787,1,0,0,0,2.133250e+07,304.0,311.0,536.0,759,2,...,0.53000,40,20.0,20,0,1,0,0,0,0
9788,1,1,1,1,3.257303e+05,108.0,46.0,0.0,759,3,...,0.58817,0,0.0,0,0,0,1,0,0,1
9789,1,1,1,1,0.000000e+00,142.0,48.0,46.0,760,1,...,0.63750,4,4.0,4,0,1,0,0,0,0


In [ ]:
df_train.dtypes


Pricing, Delivery_Terms_Quote_Appr             int64
Pricing, Delivery_Terms_Approved               int64
Bureaucratic_Code_0_Approval                   int64
Bureaucratic_Code_0_Approved                   int64
Month                                 datetime64[ns]
Total_Taxable_Amount_(converted)             float64
Delivery_Delay                               float64
duration                                     float64
timeInSystem                                 float64
Opportunity_Created_Date_delta                 int32
Products_Quantity                              int64
ASP sum                                      float64
ASP mean                                     float64
Total TRF                                      int64
TRF mean                                     float64
TRF max                                        int64
Region APAC                                    int64
Region Americas                                int64
Region EMEA                                   

In [ ]:
toTrain.dtypes

Pricing, Delivery_Terms_Quote_Appr      int64
Pricing, Delivery_Terms_Approved        int64
Bureaucratic_Code_0_Approval            int64
Bureaucratic_Code_0_Approved            int64
Total_Taxable_Amount_(converted)      float64
Delivery_Delay                        float64
duration                              float64
timeInSystem                          float64
Opportunity_Created_Date_delta          int32
Products_Quantity                       int64
ASP sum                               float64
ASP mean                              float64
Total TRF                               int64
TRF mean                              float64
TRF max                                 int64
Region APAC                             int64
Region Americas                         int64
Region EMEA                             int64
Region Japan                            int64
Region Middle East                      int64
Target                                  int64
dtype: object

In [ ]:

df_test

,"Pricing, Delivery_Terms_Quote_Appr","Pricing, Delivery_Terms_Approved",Bureaucratic_Code_0_Approval,Bureaucratic_Code_0_Approved,Month,Total_Taxable_Amount_(converted),Delivery_Delay,duration,timeInSystem,Opportunity_Created_Date_delta,...,ASP sum,ASP mean,Total TRF,TRF mean,TRF max,Region APAC,Region Americas,Region EMEA,Region Japan,Region Middle East
0,1,1,1,1,2019-05-01,4.159550e+05,19.0,0.0,646.0,1984,...,1.04061,0.346870,0,0.0,0,0,0,1,0,0
1,1,1,1,1,2019-06-01,8.428826e+05,50.0,13.0,646.0,1984,...,1.87762,0.375524,1,0.2,1,0,0,1,0,0
2,1,1,0,0,2019-10-01,2.103750e+04,172.0,4.0,55.0,1984,...,0.42500,0.425000,0,0.0,0,0,1,0,0,0
3,1,1,1,0,2019-12-01,2.169106e+06,242.0,4.0,2085.0,1984,...,2.55000,0.425000,6,1.0,1,0,1,0,0,0
4,0,0,0,0,2019-05-01,5.752500e+03,20.0,1.0,225.0,1984,...,0.39000,0.390000,0,0.0,0,0,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1562,1,1,0,0,2019-05-01,1.477500e+05,5.0,0.0,683.0,1998,...,0.60000,0.300000,0,0.0,0,0,0,1,0,0
1563,1,1,0,0,2019-10-01,4.505490e+04,158.0,1.0,322.0,1998,...,0.41000,0.410000,0,0.0,0,0,1,0,0,0
1564,1,1,1,1,2019-10-01,1.001220e+05,158.0,1.0,322.0,1998,...,0.41000,0.410000,0,0.0,0,0,1,0,0,0
1565,1,1,0,0,2019-08-01,1.432200e+05,97.0,0.0,240.0,1998,...,0.50000,0.500000,0,0.0,0,0,1,0,0,0


In [ ]:
df_train

,"Pricing, Delivery_Terms_Quote_Appr","Pricing, Delivery_Terms_Approved",Bureaucratic_Code_0_Approval,Bureaucratic_Code_0_Approved,Month,Total_Taxable_Amount_(converted),Delivery_Delay,duration,timeInSystem,Opportunity_Created_Date_delta,...,ASP mean,Total TRF,TRF mean,TRF max,Region APAC,Region Americas,Region EMEA,Region Japan,Region Middle East,Target
0,1,1,1,1,2016-05-01,5.941391e+06,146.0,189.0,174.0,762,...,0.58817,10,10.0,10,0,0,1,0,0,0
1,0,0,0,0,2016-01-01,5.263822e+04,42.0,39.0,174.0,762,...,0.59948,0,0.0,0,0,0,1,0,0,1
2,0,0,0,0,2016-01-01,8.386560e+04,48.0,296.0,231.0,763,...,0.48000,0,0.0,0,0,1,0,0,0,1
3,1,0,1,0,2018-02-01,7.421882e+06,786.0,840.0,864.0,763,...,0.53000,14,14.0,14,0,1,0,0,0,0
4,1,0,1,0,2018-02-01,1.335719e+07,786.0,840.0,864.0,763,...,0.53000,25,25.0,25,0,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9786,1,1,0,0,2016-04-01,4.592234e+05,119.0,119.0,860.0,759,...,0.58251,1,1.0,1,0,0,1,0,0,1
9787,1,0,0,0,2016-10-01,2.133250e+07,304.0,311.0,536.0,759,...,0.53000,40,20.0,20,0,1,0,0,0,0
9788,1,1,1,1,2016-03-01,3.257303e+05,108.0,46.0,0.0,759,...,0.58817,0,0.0,0,0,0,1,0,0,1
9789,1,1,1,1,2016-04-01,0.000000e+00,142.0,48.0,46.0,760,...,0.63750,4,4.0,4,0,1,0,0,0,0


In [ ]:
print(df_train.shape, df_test.shape) #shapes del df originales (16947, 52) (2551, 51)

(9791, 22) (1567, 21)


In [ ]:
print(toTrain.shape, toTrain.shape) #shapes del df originales (16947, 52) (2551, 51)

In [ ]:
df_test

,"Pricing, Delivery_Terms_Quote_Appr","Pricing, Delivery_Terms_Approved",Bureaucratic_Code_0_Approval,Bureaucratic_Code_0_Approved,Month,Total_Taxable_Amount_(converted),Delivery_Delay,duration,timeInSystem,Opportunity_Created_Date_delta,...,ASP sum,ASP mean,Total TRF,TRF mean,TRF max,Region APAC,Region Americas,Region EMEA,Region Japan,Region Middle East
0,1,1,1,1,2019-05-01,4.159550e+05,19.0,0.0,646.0,1984,...,1.04061,0.346870,0,0.0,0,0,0,1,0,0
1,1,1,1,1,2019-06-01,8.428826e+05,50.0,13.0,646.0,1984,...,1.87762,0.375524,1,0.2,1,0,0,1,0,0
2,1,1,0,0,2019-10-01,2.103750e+04,172.0,4.0,55.0,1984,...,0.42500,0.425000,0,0.0,0,0,1,0,0,0
3,1,1,1,0,2019-12-01,2.169106e+06,242.0,4.0,2085.0,1984,...,2.55000,0.425000,6,1.0,1,0,1,0,0,0
4,0,0,0,0,2019-05-01,5.752500e+03,20.0,1.0,225.0,1984,...,0.39000,0.390000,0,0.0,0,0,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1562,1,1,0,0,2019-05-01,1.477500e+05,5.0,0.0,683.0,1998,...,0.60000,0.300000,0,0.0,0,0,0,1,0,0
1563,1,1,0,0,2019-10-01,4.505490e+04,158.0,1.0,322.0,1998,...,0.41000,0.410000,0,0.0,0,0,1,0,0,0
1564,1,1,1,1,2019-10-01,1.001220e+05,158.0,1.0,322.0,1998,...,0.41000,0.410000,0,0.0,0,0,1,0,0,0
1565,1,1,0,0,2019-08-01,1.432200e+05,97.0,0.0,240.0,1998,...,0.50000,0.500000,0,0.0,0,0,1,0,0,0


In [ ]:
df_train

,"Pricing, Delivery_Terms_Quote_Appr","Pricing, Delivery_Terms_Approved",Bureaucratic_Code_0_Approval,Bureaucratic_Code_0_Approved,Month,Total_Taxable_Amount_(converted),Delivery_Delay,duration,timeInSystem,Opportunity_Created_Date_delta,...,ASP mean,Total TRF,TRF mean,TRF max,Region APAC,Region Americas,Region EMEA,Region Japan,Region Middle East,Target
0,1,1,1,1,2016-05-01,5.941391e+06,146.0,189.0,174.0,762,...,0.58817,10,10.0,10,0,0,1,0,0,0
1,0,0,0,0,2016-01-01,5.263822e+04,42.0,39.0,174.0,762,...,0.59948,0,0.0,0,0,0,1,0,0,1
2,0,0,0,0,2016-01-01,8.386560e+04,48.0,296.0,231.0,763,...,0.48000,0,0.0,0,0,1,0,0,0,1
3,1,0,1,0,2018-02-01,7.421882e+06,786.0,840.0,864.0,763,...,0.53000,14,14.0,14,0,1,0,0,0,0
4,1,0,1,0,2018-02-01,1.335719e+07,786.0,840.0,864.0,763,...,0.53000,25,25.0,25,0,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9786,1,1,0,0,2016-04-01,4.592234e+05,119.0,119.0,860.0,759,...,0.58251,1,1.0,1,0,0,1,0,0,1
9787,1,0,0,0,2016-10-01,2.133250e+07,304.0,311.0,536.0,759,...,0.53000,40,20.0,20,0,1,0,0,0,0
9788,1,1,1,1,2016-03-01,3.257303e+05,108.0,46.0,0.0,759,...,0.58817,0,0.0,0,0,0,1,0,0,1
9789,1,1,1,1,2016-04-01,0.000000e+00,142.0,48.0,46.0,760,...,0.63750,4,4.0,4,0,1,0,0,0,0


In [ ]:
print(df_train.shape, df_test.shape) #shapes del df originales (16947, 52) (2551, 51)

(9791, 22) (1567, 21)


In [ ]:
X, y = toTrain.iloc[:,:-1],toTrain.iloc[:,-1]


In [ ]:
# split into train test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=1)
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

(6559, 20) (3232, 20) (6559,) (3232,)


In [ ]:
# fit the model
model = RandomForestRegressor(random_state=1, n_estimators=100)
model.fit(X_train, y_train)

RandomForestRegressor(random_state=1)

In [ ]:
# make predictions
yhat = model.predict(X_test)


In [ ]:
# evaluate predictions
mae = mean_absolute_error(y_test, yhat)
print('MAE: %.3f' % mae)

MAE: 0.251


In [ ]:
from sklearn.metrics import log_loss


In [ ]:
logloss = log_loss(y_test, yhat)
print("Log Loss: %f" % (logloss))
#print(max(logloss), min(logloss))

Log Loss: 0.406500


In [ ]:
#prediccion del set de test

In [ ]:
import numpy as np

## MEJORA DE HIPERPARAMETROS
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from scipy.stats import randint as sp_randint

param_dist_random = {
    "max_depth": [5, None],
    "max_features": sp_randint(1, 11),
    "min_samples_split": sp_randint(2, 11),
    "min_samples_leaf": sp_randint(1, 11),
    "bootstrap": [True, False],
    "n_estimators": np.linspace(10,1000,10).astype(int)
   
}
param_busqueda_rf = {
    "criterion": ["gini", "entropy"],
    "n_estimators": np.linspace(10,1000,10).astype(int),
    "class_weight": [None, "balanced"]
}

In [ ]:
model_rfr = RandomForestRegressor()

clf = RandomizedSearchCV(estimator=model_rfr, 
                         param_distributions=param_dist_random,
                         n_jobs=3, n_iter=100, random_state=7)

In [ ]:
clf.fit(X_train,y_train)

RandomizedSearchCV(estimator=RandomForestRegressor(), n_iter=100, n_jobs=3,
                   param_distributions={'bootstrap': [True, False],
                                        'max_depth': [5, None],
                                        'max_features': <scipy.stats._distn_infrastructure.rv_frozen object at 0x0000016576BC9700>,
                                        'min_samples_leaf': <scipy.stats._distn_infrastructure.rv_frozen object at 0x000001657587AE80>,
                                        'min_samples_split': <scipy.stats._distn_infrastructure.rv_frozen object at 0x000001657520C040>,
                                        'n_estimators': array([  10,  120,  230,  340,  450,  560,  670,  780,  890, 1000])},
                   random_state=7, scoring='roc_auc')

In [ ]:
print(clf.best_score_)

print(clf.best_estimator_)

0.9098858591179934
RandomForestRegressor(max_features=3, min_samples_split=5, n_estimators=1000)


In [ ]:
from sklearn.model_selection import cross_validate

def evaluar_modelo(estimador, X, y):
    resultados_estimador = cross_validate(estimador, X, y,
                                          scoring="neg_log_loss", n_jobs=-1, 
                                          cv=10, return_train_score=True)
    return resultados_estimador

resultados = {}

def ver_resultados():
    resultados_df  = pd.DataFrame(resultados).T
    resultados_cols = resultados_df.columns
    for col in resultados_df:
        resultados_df[col] = resultados_df[col].apply(np.mean)
        resultados_df[col+"_idx"] = resultados_df[col] / resultados_df[col].min()
    return resultados_df

In [ ]:
best_model = RandomForestRegressor(bootstrap=False, max_features=3, min_samples_split=6,
                      n_estimators=890)

In [ ]:
X_train

,"Pricing, Delivery_Terms_Quote_Appr","Pricing, Delivery_Terms_Approved",Bureaucratic_Code_0_Approval,Bureaucratic_Code_0_Approved,Total_Taxable_Amount_(converted),Delivery_Delay,duration,timeInSystem,Opportunity_Created_Date_delta,Products_Quantity,ASP sum,ASP mean,Total TRF,TRF mean,TRF max,Region APAC,Region Americas,Region EMEA,Region Japan,Region Middle East
8107,0,0,0,0,255855.600000,81.0,79.0,748.0,1694,1,0.33000,0.33000,1,1.0,1,1,0,0,0,0
1793,0,0,0,0,549820.430000,405.0,375.0,1060.0,959,1,0.38500,0.38500,1,1.0,1,0,1,0,0,0
5761,0,0,0,0,341700.000000,104.0,289.0,406.0,1388,1,0.42500,0.42500,1,1.0,1,0,1,0,0,0
2089,1,0,0,0,3125.033501,63.0,231.0,466.0,997,1,0.61135,0.61135,0,0.0,0,0,0,0,1,0
4484,1,1,1,1,791700.000000,109.0,147.0,1091.0,1409,1,0.39000,0.39000,2,2.0,2,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2895,0,0,0,0,0.000000,4.0,104.0,1202.0,1101,1,0.42000,0.42000,0,0.0,0,0,1,0,0,0
7813,1,0,1,0,165204.720000,116.0,125.0,1755.0,1654,1,0.36195,0.36195,0,0.0,0,0,0,1,0,0
905,0,0,0,0,399000.000000,426.0,441.0,1022.0,1212,1,0.40000,0.40000,1,1.0,1,1,0,0,0,0
5192,1,1,0,0,8109.618048,25.0,223.0,802.0,1333,8,0.58437,0.58437,0,0.0,0,0,0,0,1,0


In [ ]:
best_model = best_model.fit(X_train,y_train)

In [ ]:
preds = best_model.predict(toTest)

In [ ]:
preds

array([0.83694757, 0.67016854, 0.62205993, ..., 0.61953184, 0.66013109,
       0.36331461])

In [ ]:
toTest

,"Pricing, Delivery_Terms_Quote_Appr","Pricing, Delivery_Terms_Approved",Bureaucratic_Code_0_Approval,Bureaucratic_Code_0_Approved,Total_Taxable_Amount_(converted),Delivery_Delay,duration,timeInSystem,Opportunity_Created_Date_delta,Products_Quantity,ASP sum,ASP mean,Total TRF,TRF mean,TRF max,Region APAC,Region Americas,Region EMEA,Region Japan,Region Middle East
0,1,1,1,1,4.159550e+05,19.0,0.0,646.0,1984,3,1.04061,0.346870,0,0.0,0,0,0,1,0,0
1,1,1,1,1,8.428826e+05,50.0,13.0,646.0,1984,5,1.87762,0.375524,1,0.2,1,0,0,1,0,0
2,1,1,0,0,2.103750e+04,172.0,4.0,55.0,1984,1,0.42500,0.425000,0,0.0,0,0,1,0,0,0
3,1,1,1,0,2.169106e+06,242.0,4.0,2085.0,1984,6,2.55000,0.425000,6,1.0,1,0,1,0,0,0
4,0,0,0,0,5.752500e+03,20.0,1.0,225.0,1984,1,0.39000,0.390000,0,0.0,0,0,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1562,1,1,0,0,1.477500e+05,5.0,0.0,683.0,1998,2,0.60000,0.300000,0,0.0,0,0,0,1,0,0
1563,1,1,0,0,4.505490e+04,158.0,1.0,322.0,1998,1,0.41000,0.410000,0,0.0,0,0,1,0,0,0
1564,1,1,1,1,1.001220e+05,158.0,1.0,322.0,1998,1,0.41000,0.410000,0,0.0,0,0,1,0,0,0
1565,1,1,0,0,1.432200e+05,97.0,0.0,240.0,1998,1,0.50000,0.500000,0,0.0,0,0,1,0,0,0


In [ ]:
ops = ids
out = pd.DataFrame()
out["Opportunity_ID"] = ops
out["Target"] = preds

out.to_csv(r"resultados_rfr_v5.csv" , index=False)

In [ ]:
pd.DataFrame({'Opportunity_ID': df_test.index, 'Target': preds}).to_csv('test_cesar_rfr_v3.csv', index=False, columns=['Opportunity_ID', 'label'])